# Project 1

Project 1 is about classification on text data, where you extract features from raw texts and try different classification approaches to classify them into topics.

During this discussion, we introduce some key concepts and usages of relevant python packages to help you get started with your project.

This explanation is mainly from different sections of the scikit-learn tutorial on text classification available at http://scikit-learn.org.

### A short introduction to `Numpy`

In [1]:
import numpy as np
a = [1,2,3]
print(a)

[1, 2, 3]


In [2]:
a = np.array(['hello', 'worlds']) # this a 1D array
b = np.array([1, 2, 3])
c = np.array([[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]]) # this is a 2D array
print(a)
print('-' * 20)
print(b)
print('-' * 20)
print(c)
print('-' * 20)
print(a.dtype, b.dtype, c.dtype)
print('-' * 20)
print(c.shape) # same member variable exists for Pandas it is shape, not shape()

['hello' 'worlds']
--------------------
[1 2 3]
--------------------
[[1. 2. 3.]
 [4. 5. 6.]]
--------------------
<U6 int64 float64
--------------------
(2, 3)


In [3]:
d = np.random.rand(3,4) # Project 1 see GLoVe part
d
#type(d)

array([[0.36167018, 0.33628929, 0.29007335, 0.06613926],
       [0.15653837, 0.14225992, 0.00925611, 0.027723  ],
       [0.90946097, 0.39310642, 0.10147884, 0.54801228]])

In [4]:
d[:,1:3]

array([[0.33628929, 0.29007335],
       [0.14225992, 0.00925611],
       [0.39310642, 0.10147884]])

In [5]:
d[0,:2]

array([0.36167018, 0.33628929])

In [6]:
d + 10 # braodcasting
a = [i + 10 for i in [1,2,4]] # don't recommend
a

[11, 12, 14]

In [7]:
e = np.ones([3, 2])
f = np.zeros(e.shape)
f

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [8]:
np.vstack([e, f])

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [9]:
np.hstack([e, f])

array([[1., 1., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 0., 0.]])

In [10]:
np.sum(e)

6.0

In [11]:
np.sum(e, axis=0) # summation across rows is axis = 0 and across col is axis = 1

array([3., 3.])

In [12]:
np.sum(e, axis=1)

array([2., 2., 2.])

In [13]:
g = np.array([[1, 2], [3, 4], [5, 6]])
g

array([[1, 2],
       [3, 4],
       [5, 6]])

In [14]:
e + g

array([[2., 3.],
       [4., 5.],
       [6., 7.]])

In [15]:
h = np.array([[1],[1]])
h

array([[1],
       [1]])

In [16]:
np.matmul(g, h)

array([[ 3],
       [ 7],
       [11]])

### Dataset
1. In the discussion, we work with “20 Newsgroups” dataset as an alternative to the custom dataset. Many of the functions will be the same. It is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups, each corresponding to a different topic.
2. To manually load the data, you need to run this python code. <a href="https://www.dropbox.com/s/5oek8qbsge1y64b/fetch_data.py?dl=0">link to fetch_data.py</a>
3. Easiest way to load the data is to use the built-in dataset loader for 20 newsgroups from scikit-learn package.


In [17]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'comp.sys.mac.hardware']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [18]:
print(type(twenty_train))
print(twenty_train.keys())

<class 'sklearn.utils._bunch.Bunch'>
dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [19]:
print(twenty_train.data[0])

From: winstead@faraday.ece.cmu.edu (Charles Holden Winstead)
Subject: ftp site for Radius software???
Organization: Electrical and Computer Engineering, Carnegie Mellon

Hey All,

Does anyone know if I can ftp to get the newest version of Radiusware
and soft pivot from Radius?  I bought a pivot monitor, but it has an
old version of this software and won't work on my C650, and Radius said
it would be 4-5 weeks until delivery.

Thanks!

-Chuck





In [20]:
# list of category indices of the documents
twenty_train.target # twenty_train['target']

array([1, 0, 0, ..., 0, 1, 0])

In [21]:
# so index 0 corresponds to 'comp.graphics'; 1 to 'comp.sys.mac.hardware'
print(twenty_train.target_names) # twenty_train['target_names'] # instead look at the column titled root_label

['comp.graphics', 'comp.sys.mac.hardware']


In [22]:
print(twenty_train.target_names[twenty_train.target[0]]) # unique() in pandas

comp.sys.mac.hardware


The files themselves are loaded in memory in the data attribute.

In [23]:
print(len(twenty_train.data)) # shape of a pandas dataframe is very useful especially in DL when you might lose control of your matrix dimensions
print(len(twenty_train.filenames))
print(len(twenty_train.target))
print(len(twenty_train.target_names))

1162
1162
1162
2


## Extracting features from text files

### CountVectorizer
Convert a collection of text documents to a matrix of token counts

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1) # np.array is a class
vectorizer

CountVectorizer()

For the detailed documentation of `CountVectorizer`, see

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

For regular expression:
https://docs.python.org/2/library/re.html

#### Stop words

Words that are too common to be useful in classification

In [25]:
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS
print(stop_words)
print(len(stop_words))

frozenset({'yourself', 'yet', 'itself', 'by', 'whatever', 'enough', 'sometimes', 'why', 'least', 'has', 'nevertheless', 'into', 'themselves', 'get', 'else', 'anyway', 'have', 'are', 'move', 'under', 'twenty', 'thereby', 'although', 'wherein', 'since', 'keep', 'to', 'indeed', 'here', 'part', 'several', 'its', 'becoming', 'up', 'hundred', 'which', 'sometime', 'or', 'among', 'hasnt', 'around', 'co', 'fire', 'whereas', 'you', 'until', 'behind', 'again', 'always', 'ourselves', 'four', 'thereupon', 'one', 'your', 'were', 'amount', 'third', 'while', 'who', 'empty', 'nothing', 'there', 'they', 'meanwhile', 'whose', 'might', 'couldnt', 'been', 'any', 'noone', 'became', 'alone', 'afterwards', 'he', 'and', 'hereupon', 'ever', 'side', 'con', 'that', 'perhaps', 'sincere', 'most', 'what', 'may', 'once', 'never', 'elsewhere', 'via', 'describe', 'sixty', 'all', 'show', 'top', 'same', 'inc', 'everyone', 'on', 'name', 'not', 'somewhere', 'amoungst', 'then', 'three', 'therefore', 'twelve', 'due', 'either

In [26]:
corpus = [
    'This is the first document. I was went to store.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'Arash Shadi and Pavan'
]
X = vectorizer.fit_transform(corpus) # you need a batch from the corpus to extract reasonable features especially in memory-less settings.
X 

<5x16 sparse matrix of type '<class 'numpy.int64'>'
	with 27 stored elements in Compressed Sparse Row format>

In [27]:
# feature names
vectorizer.get_feature_names()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [ ]:
X.toarray()

The converse mapping from feature name to column index is stored in the vocabulary_ attribute of the vectorizer:

In [ ]:
# count_vect = CountVectorizer()
#count_vect = CountVectorizer(stop_words='english')
count_vect = CountVectorizer(stop_words='english', min_df=3, max_df=0.7)
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

In [ ]:
X_test_counts = count_vect.transform(twenty_test.data)
X_test_counts.shape

In [ ]:
print(count_vect.get_feature_names()[:20])
print('-' * 20)
print(count_vect.get_feature_names()[5161])
print(count_vect.vocabulary_.get('circuitry'))

### TFIDF

$$TF\times IDF(t,d)=tf(t,d)\times idf(t)$$
<hr>
$$idf(t)=\log(\frac{n}{df(t)})+1$$

- $tf(t, d)$: term frequency of term $t$ in the document $d$.


- $idf(t)$: inverse document frequency of term $t$ across the document dataset.
  - $df(t)$: # of documents that contain the term $t$.
  - Intuition: words that appear in all documents are useless in classificaiton.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)
print('-' * 20)
print(X_train_counts.toarray()[:30,:5])
print('-' * 20)
print(X_train_tfidf.toarray()[:30,:5])

### Training a classifier

Let's train a classifier to predict the category of a post.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [ ]:
docs_new = ['He is an OS developer', 'OpenGL on the GPU is fast', 'I like Apple Mojave OS', "This discussion was full of weird stuff."]

X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

### Dimensionality reduction

In [ ]:
from sklearn.decomposition import NMF

model = NMF(n_components=50, init='random', random_state=0)
W_train = model.fit_transform(X_train_tfidf)

In [ ]:
print(W_train.shape)

In [ ]:
H = model.components_
H.shape

In [ ]:
H

In [ ]:
W_test = model.transform(X_new_tfidf)

clf = MultinomialNB().fit(W_train, twenty_train.target)


predicted = clf.predict(W_test)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

In [ ]:
W_test

In [ ]:
W_test.shape